20180634_최준혁_과제5번

1. (4 points) Find an appropriate GARCH model for the KOSPI index returns
during the period 2021:01:02-2021:12:30 (on the basis of daily, closing prices). Examine whether the coefficient estimates are significant at the conventional significance levels.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa import stattools
import pandas_datareader as wb
import datetime
from arch import arch_model

### 데이터 로드

In [2]:
s = datetime.datetime(2021,1,2)
e = datetime.datetime(2021,12,30)
data = wb.DataReader("^KS11",'yahoo',s,e)
df = data['Close'].dropna()

In [3]:
df

Date
2021-01-04    2944.449951
2021-01-05    2990.570068
2021-01-06    2968.209961
2021-01-07    3031.679932
2021-01-08    3152.179932
                 ...     
2021-12-24    3012.429932
2021-12-27    2999.550049
2021-12-28    3020.239990
2021-12-29    2993.290039
2021-12-30    2977.649902
Name: Close, Length: 248, dtype: float64

### Stationary Check

In [4]:
from statsmodels.tsa.stattools import adfuller

adfuller(df, regression='c') #Non-stationary

(-2.4125118905833296,
 0.13823611562460908,
 0,
 247,
 {'1%': -3.457105309726321,
  '5%': -2.873313676101283,
  '10%': -2.5730443824681606},
 2209.7981086363407)

In [5]:
log_return = [] #log_return은 log difference
for j in range(len(df)):
    if j == 0 : continue
    else : log_return.append(np.log(df[j])-np.log(df[j-1])) 
log_return = pd.Series(log_return)

In [6]:
log_return

0      0.015542
1     -0.007505
2      0.021158
3      0.038977
4     -0.001184
         ...   
242    0.004745
243   -0.004285
244    0.006874
245   -0.008963
246   -0.005239
Length: 247, dtype: float64

In [7]:
adfuller(log_return, regression='c')
#log_return은 p-value = 매우 낮음 -> stationary, unit root가 없다. 즉 log return 사용

(-10.507441759062361,
 1.0453766982299086e-18,
 2,
 244,
 {'1%': -3.457437824930831,
  '5%': -2.873459364726563,
  '10%': -2.573122099570008},
 -1489.533956433947)

#### 번외 : pct_change로 stationary하게 만들기

In [8]:
pct_df = df.pct_change().dropna() #차이를 백분율화 시킨 것

adfuller(pct_df, regression='c') #이것도 stationary한 자료임

(-10.516760298174889,
 9.918102900374887e-19,
 2,
 244,
 {'1%': -3.457437824930831,
  '5%': -2.873459364726563,
  '10%': -2.573122099570008},
 -1489.9264963230776)

### Find appropriate GARCH and Examine the coefficients

In [9]:
import itertools

p=q = range(0,10)
pq = list(itertools.product(p,q))
mean = ['Constant', 'Zero', 'LS', 'AR', 'ARX', 'HAR', 'HARX']

In [10]:
best_bic = np.inf 
best_pq = None
best_mean = None
top_model = None
best_mdl = None

for param in pq:
    for m in mean:
        try: 
            tmp_mdl = arch_model(log_return, mean= m , vol='GARCH', p=param[0], q=param[1])
            res = tmp_mdl.fit()
            
            print("GARCH{}x{}: - mean: {} - bic:{}".format(pq[0],pq[1],m,res.bic))
            if res.bic < best_bic:
                best_bic = res.bic
                best_pq = param
                best_mean = m
                best_mdl = tmp_mdl
                print("OK")
        except : 
            print("unexpecter error")
            continue
print("------------------------------BEST--------------------------")
print("GARCH{}x{}: - mean: {} - bic:{}".format(best_pq[0],best_pq[1], best_mean, best_bic))

unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter error
unexpecter err

C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001042. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001042. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: Data

Iteration:      6,   Func. Count:     56,   Neg. LLF: 886373.3086182835
Iteration:      7,   Func. Count:     67,   Neg. LLF: 19630.89012543332
Iteration:      8,   Func. Count:     77,   Neg. LLF: 17105.617189287586
Iteration:      9,   Func. Count:     86,   Neg. LLF: -794.5034963289088
Optimization terminated successfully    (Exit mode 0)
            Current function value: -794.5034954126315
            Iterations: 13
            Function evaluations: 86
            Gradient evaluations: 9
GARCH(0, 0)x(0, 1): - mean: AR - bic:-1572.4788258153792
Iteration:      1,   Func. Count:      5,   Neg. LLF: 3006466.624989305
Iteration:      2,   Func. Count:     16,   Neg. LLF: 15969.872384851013
Iteration:      3,   Func. Count:     26,   Neg. LLF: 13763599.449762307
Iteration:      4,   Func. Count:     37,   Neg. LLF: 714.8934215453639
Iteration:      5,   Func. Count:     45,   Neg. LLF: 563809731.9635264
Iteration:      6,   Func. Count:     56,   Neg. LLF: 886373.3086182835
Iteration:

C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001042. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001042. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: Data

Iteration:      5,   Func. Count:     51,   Neg. LLF: -806.06060934316
Optimization terminated successfully    (Exit mode 0)
            Current function value: -806.0606097079244
            Iterations: 9
            Function evaluations: 51
            Gradient evaluations: 5
GARCH(0, 0)x(0, 1): - mean: AR - bic:-1590.083666069337
Iteration:      1,   Func. Count:      6,   Neg. LLF: 2.287427994341976e+17
Iteration:      2,   Func. Count:     18,   Neg. LLF: 4803739.5674252
Iteration:      3,   Func. Count:     30,   Neg. LLF: 22144127135663.227
Iteration:      4,   Func. Count:     41,   Neg. LLF: 19378.710993172677
Iteration:      5,   Func. Count:     51,   Neg. LLF: -806.06060934316
Optimization terminated successfully    (Exit mode 0)
            Current function value: -806.0606097079244
            Iterations: 9
            Function evaluations: 51
            Gradient evaluations: 5
GARCH(0, 0)x(0, 1): - mean: ARX - bic:-1590.083666069337
Iteration:      1,   Func. Count:    

...

Iteration:      3,   Func. Count:     74,   Neg. LLF: 11034418330.852674
Iteration:      4,   Func. Count:    101,   Neg. LLF: -799.9457528707367
Optimization terminated successfully    (Exit mode 0)
            Current function value: -799.9457539319321
            Iterations: 8
            Function evaluations: 101
            Gradient evaluations: 4
GARCH(0, 0)x(0, 1): - mean: HARX - bic:-1495.2131294679327
Iteration:      1,   Func. Count:     22,   Neg. LLF: 56577949.002739534
Iteration:      2,   Func. Count:     50,   Neg. LLF: 3545119473527093.5
Iteration:      3,   Func. Count:     78,   Neg. LLF: -799.7569169079034
Optimization terminated successfully    (Exit mode 0)
            Current function value: -799.7569180567423
            Iterations: 7
            Function evaluations: 78
            Gradient evaluations: 3
GARCH(0, 0)x(0, 1): - mean: Constant - bic:-1489.326069380925
Iteration:      1,   Func. Count:     21,   Neg. LLF: 117582770.08054173
Iteration:      2,   Fun

C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001042. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001042. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


Iteration:     10,   Func. Count:    218,   Neg. LLF: -770.6991587826562
Iteration:     11,   Func. Count:    239,   Neg. LLF: -795.8041621810571
Iteration:     12,   Func. Count:    260,   Neg. LLF: -806.445054194276
Iteration:     13,   Func. Count:    281,   Neg. LLF: -808.5455610084547
Iteration:     14,   Func. Count:    302,   Neg. LLF: -808.2127903404933
Iteration:     15,   Func. Count:    323,   Neg. LLF: -802.7604478352956
Iteration:     16,   Func. Count:    344,   Neg. LLF: -806.1144913041023
Iteration:     17,   Func. Count:    366,   Neg. LLF: -809.0448934244152
Iteration:     18,   Func. Count:    386,   Neg. LLF: -809.2383711808438
Iteration:     19,   Func. Count:    406,   Neg. LLF: -809.3030612467455
Iteration:     20,   Func. Count:    426,   Neg. LLF: -809.4492097886546
Iteration:     21,   Func. Count:    446,   Neg. LLF: -809.4577468682521
Iteration:     22,   Func. Count:    466,   Neg. LLF: -809.4707995166268
Iteration:     23,   Func. Count:    486,   Neg. LLF

C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001042. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001042. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: Data

Iteration:      2,   Func. Count:     50,   Neg. LLF: 3545119473527093.5
Iteration:      3,   Func. Count:     78,   Neg. LLF: -799.7569169079034
Optimization terminated successfully    (Exit mode 0)
            Current function value: -799.7569180567423
            Iterations: 7
            Function evaluations: 78
            Gradient evaluations: 3
GARCH(0, 0)x(0, 1): - mean: AR - bic:-1489.326069380925
Iteration:      1,   Func. Count:     22,   Neg. LLF: 56577949.002739534
Iteration:      2,   Func. Count:     50,   Neg. LLF: 3545119473527093.5
Iteration:      3,   Func. Count:     78,   Neg. LLF: -799.7569169079034
Optimization terminated successfully    (Exit mode 0)
            Current function value: -799.7569180567423
            Iterations: 7
            Function evaluations: 78
            Gradient evaluations: 3
GARCH(0, 0)x(0, 1): - mean: ARX - bic:-1489.326069380925
Iteration:      1,   Func. Count:     22,   Neg. LLF: 56577949.002739534
Iteration:      2,   Func. Count:

In [11]:
model = arch_model(log_return, mean=best_mean, vol='GARCH', p=best_pq[0], q=best_pq[1])
model_fit = model.fit()
model_fit.summary()

Iteration:      1,   Func. Count:      4,   Neg. LLF: -806.0201041040168
Optimization terminated successfully    (Exit mode 0)
            Current function value: -806.0201044683258
            Iterations: 5
            Function evaluations: 4
            Gradient evaluations: 1


C:\Users\chris\anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001042. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


<class 'statsmodels.iolib.summary.Summary'>
"""
                       Zero Mean - GARCH Model Results                        
==============================================================================
Dep. Variable:                   None   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.004
Vol Model:                      GARCH   Log-Likelihood:                806.020
Distribution:                  Normal   AIC:                          -1606.04
Method:            Maximum Likelihood   BIC:                          -1595.51
                                        No. Observations:                  247
Date:                Thu, Nov 17 2022   Df Residuals:                      247
Time:                        03:36:45   Df Model:                            0
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      1.0418e-05  3.708e-11  2.810e+05      0.000 [1.042e-05,1.042e-05]
alpha[1]       0.2000  7.265e-02      2.753  5.905e-03   [5.761e-02,  0.342]
beta[1]        0.7000  5.311e-02     13.181  1.132e-39     [  0.596,  0.804]
============================================================================

Covariance estimator: robust
"""

- Zero Mean의GARCH(1,1)가 kospi의 Log_return에 대해서 가장 적절한 모형이다
- 계수들을 통상적인 유의수준으로 보면 모든 계수가 유의하다